# limpiesa y lectura de datos


In [4]:
from pathlib import Path
import pandas as pd
import numpy as np
import re

DATA_DIR = Path("csv").resolve()
assert DATA_DIR.exists(), f"No existe {DATA_DIR}"

# Detectar archivos
asistencia_files = sorted(DATA_DIR.glob("Asistencia_Anual_*.csv"))
rend_files = sorted(DATA_DIR.glob("Rendimiento_*.csv"))

print("🗂️ Asistencia:", [f.name for f in asistencia_files])
print("🗂️ Rendimiento:", [f.name for f in rend_files])

# Helper robusto de lectura
def read_csv_smart(path: Path, nrows=None):
    try:
        return pd.read_csv(path, nrows=nrows)
    except UnicodeDecodeError:
        return pd.read_csv(path, nrows=nrows, encoding="latin-1")

🗂️ Asistencia: ['Asistencia_Anual_2022.csv', 'Asistencia_Anual_2023.csv', 'Asistencia_Anual_2024.csv']
🗂️ Rendimiento: ['Rendimiento_2022_2023.csv', 'Rendimiento_2023_2024.csv', 'Rendimiento_2024_2025.csv']


In [ ]:
def read_csv_smart(path: Path, nrows=None):
    """
    Lee CSV detectando separador y encoding. Omite líneas defectuosas.
    """
    for enc in ("utf-8", "utf-8-sig", "latin-1"):
        # 1) auto-inferencia de separador (engine=python)
        try:
            return pd.read_csv(path, nrows=nrows, sep=None, engine="python",
                               encoding=enc, on_bad_lines="skip")
        except UnicodeDecodeError:
            continue
        except pd.errors.ParserError:
            pass
        # 2) intentos explícitos de separador común
        for sep in (";", "\t", "|", ","):
            try:
                return pd.read_csv(path, nrows=nrows, sep=sep, engine="python",
                                   encoding=enc, on_bad_lines="skip")
            except Exception:
                continue
    # 3) último recurso
    return pd.read_csv(path, nrows=nrows, sep=";", engine="python",
                       encoding="latin-1", on_bad_lines="skip")
a
# Inspección rápida de columnas
for f in asistencia_files + rend_files:
    dfh = read_csv_smart(f, nrows=5)
    print(f"\n{f.name} -> {dfh.shape}")
    print(dfh.columns.tolist())
    display(dfh.head())

# Hints para mapear nombres
COLUMN_HINTS = {
    "id": ["id", "id_alumno", "idalumno", "rut", "estudiante", "id_estudiante"],
    "anio": ["anio", "año", "ano", "periodo", "year"],
    "asistencia": ["asistencia", "asistencia_%", "porc_asistencia", "asistencia_pct"],
    "nota": ["promedio", "nota_final", "nota", "gpa"],
    "aprobado": ["aprobado", "estado", "resultado"]
}

def infer_col(df, keys):
    cols = [c for c in df.columns]
    low = {c.lower(): c for c in cols}
    for k in keys:
        for lc, orig in low.items():
            if re.search(rf"\b{k}\b", lc):
                return orig
    return None
# ...existing code...


Asistencia_Anual_2022.csv -> (5, 55)
['agno', 'rbd', 'cod_com_rbd', 'cod_reg_rbd', 'nom_reg_rbd_a', 'cod_pro_rbd', 'nom_com_rbd', 'cod_deprov_rbd', 'nom_deprov_rbd', 'rural_rbd', 'slep', 'nombre_slep', 'cohorte', 'cod_depe2', 'cod_ense', 'cod_ense2', 'cod_grado', 'let_cur', 'mrun', 'gen_alu', 'fec_nac', 'dias_asistidos_3', 'dias_asistidos_4', 'dias_asistidos_5', 'dias_asistidos_6', 'dias_asistidos_8', 'dias_asistidos_7', 'dias_asistidos_9', 'dias_asistidos_10', 'dias_asistidos_11', 'dias_asistidos_12', 'dias_asistidos_anual', 'dias_trabajados_3', 'dias_trabajados_4', 'dias_trabajados_5', 'dias_trabajados_6', 'dias_trabajados_8', 'dias_trabajados_7', 'dias_trabajados_9', 'dias_trabajados_10', 'dias_trabajados_11', 'dias_trabajados_12', 'dias_trabajados_anual', 'tasa_asistencia_3', 'tasa_asistencia_4', 'tasa_asistencia_5', 'tasa_asistencia_6', 'tasa_asistencia_8', 'tasa_asistencia_7', 'tasa_asistencia_9', 'tasa_asistencia_10', 'tasa_asistencia_11', 'tasa_asistencia_12', 'tasa_asistencia

,agno,rbd,cod_com_rbd,cod_reg_rbd,nom_reg_rbd_a,cod_pro_rbd,nom_com_rbd,cod_deprov_rbd,nom_deprov_rbd,rural_rbd,...,tasa_asistencia_5,tasa_asistencia_6,tasa_asistencia_8,tasa_asistencia_7,tasa_asistencia_9,tasa_asistencia_10,tasa_asistencia_11,tasa_asistencia_12,tasa_asistencia_anual,categoria_asis_anual
0,2022,11231,5802,5,VALPO,58,LIMACHE,51,QUILLOTA,0,...,1,1,1,1,1,1,1,1,"1,00000000000000000",4
1,2022,9145,13120,13,RM,131,ÑUÑOA,133,SANTIAGO ORIENTE,0,...,,,,0,,,,,"0,00000000000000000",1
2,2022,40298,4101,4,COQ,41,LA SERENA,41,ELQUI,0,...,,,,,,1,,,"1,00000000000000000",4
3,2022,25993,13118,13,RM,131,MACUL,133,SANTIAGO ORIENTE,0,...,0.772727272727273,0.684210526315789,1,0,1,0.736842105263158,1,1,"0,88397790055248615",3
4,2022,14389,5101,5,VALPO,51,VALPARAÍSO,53,VALPARAÍSO,0,...,1,0.941176470588235,1,1,1,1,1,,"0,99342105263157898",4



Asistencia_Anual_2023.csv -> (5, 55)
['agno', 'rbd', 'cod_com_rbd', 'cod_reg_rbd', 'nom_reg_rbd_a', 'cod_pro_rbd', 'nom_com_rbd', 'cod_deprov_rbd', 'nom_deprov_rbd', 'rural_rbd', 'slep', 'nombre_slep', 'cohorte', 'cod_depe2', 'cod_ense', 'cod_ense2', 'cod_grado', 'let_cur', 'mrun', 'gen_alu', 'fec_nac', 'dias_asistidos_3', 'dias_asistidos_4', 'dias_asistidos_5', 'dias_asistidos_6', 'dias_asistidos_7', 'dias_asistidos_8', 'dias_asistidos_9', 'dias_asistidos_10', 'dias_asistidos_11', 'dias_asistidos_12', 'dias_asistidos_anual', 'dias_trabajados_3', 'dias_trabajados_4', 'dias_trabajados_5', 'dias_trabajados_6', 'dias_trabajados_7', 'dias_trabajados_8', 'dias_trabajados_9', 'dias_trabajados_10', 'dias_trabajados_11', 'dias_trabajados_12', 'dias_trabajados_anual', 'tasa_asistencia_3', 'tasa_asistencia_4', 'tasa_asistencia_5', 'tasa_asistencia_6', 'tasa_asistencia_7', 'tasa_asistencia_8', 'tasa_asistencia_9', 'tasa_asistencia_10', 'tasa_asistencia_11', 'tasa_asistencia_12', 'tasa_asistencia

,agno,rbd,cod_com_rbd,cod_reg_rbd,nom_reg_rbd_a,cod_pro_rbd,nom_com_rbd,cod_deprov_rbd,nom_deprov_rbd,rural_rbd,...,tasa_asistencia_5,tasa_asistencia_6,tasa_asistencia_7,tasa_asistencia_8,tasa_asistencia_9,tasa_asistencia_10,tasa_asistencia_11,tasa_asistencia_12,tasa_asistencia_anual,categoria_asis_anual
0,2023,14946,5802,5,VALPO,58,LIMACHE,51,QUILLOTA,0,...,1.000000,1.000000,0.888889,0.533333,1.000000,1.000000,1.000000,,"0,93548387096774188",4
1,2023,24911,13101,13,RM,131,SANTIAGO,131,SANTIAGO CENTRO,0,...,1.000000,1.000000,1.000000,1.000000,0.916667,0.882353,0.384615,,"0,87786259541984735",3
2,2023,14389,5101,5,VALPO,51,VALPARAÍSO,53,VALPARAÍSO,0,...,0.952381,0.823529,0.727273,0.900000,1.000000,1.000000,0.941176,,"0,93670886075949367",4
3,2023,14389,5101,5,VALPO,51,VALPARAÍSO,53,VALPARAÍSO,0,...,1.000000,0.642857,0.666667,0.941176,1.000000,1.000000,1.000000,,"0,93181818181818177",4
4,2023,31220,4106,4,COQ,41,VICUÑA,41,ELQUI,1,...,1.000000,1.000000,1.000000,1.000000,0.416667,0.647059,0.923077,,"0,89393939393939392",3



Asistencia_Anual_2024.csv -> (5, 55)
['agno', 'rbd', 'cod_com_rbd', 'cod_reg_rbd', 'nom_reg_rbd_a', 'cod_pro_rbd', 'nom_com_rbd', 'cod_deprov_rbd', 'nom_deprov_rbd', 'rural_rbd', 'SLEP', 'nombre_slep', 'cohorte', 'cod_depe2', 'cod_ense', 'cod_ense2', 'cod_grado', 'let_cur', 'mrun', 'gen_alu', 'fec_nac', 'dias_asistidos_4', 'dias_asistidos_3', 'dias_asistidos_5', 'dias_asistidos_6', 'dias_asistidos_7', 'dias_asistidos_8', 'dias_asistidos_9', 'dias_asistidos_10', 'dias_asistidos_11', 'dias_asistidos_12', 'dias_asistidos_anual', 'dias_trabajados_4', 'dias_trabajados_3', 'dias_trabajados_5', 'dias_trabajados_6', 'dias_trabajados_7', 'dias_trabajados_8', 'dias_trabajados_9', 'dias_trabajados_10', 'dias_trabajados_11', 'dias_trabajados_12', 'dias_trabajados_anual', 'tasa_asistencia_4', 'tasa_asistencia_3', 'tasa_asistencia_5', 'tasa_asistencia_6', 'tasa_asistencia_7', 'tasa_asistencia_8', 'tasa_asistencia_9', 'tasa_asistencia_10', 'tasa_asistencia_11', 'tasa_asistencia_12', 'tasa_asistencia

...

,﻿AGNO,RBD,DGV_RBD,NOM_RBD,COD_REG_RBD,NOM_REG_RBD_A,COD_PRO_RBD,COD_COM_RBD,NOM_COM_RBD,COD_DEPROV_RBD,...,COD_COM_ALU,NOM_COM_ALU,COD_RAMA,COD_SEC,COD_ESPE,PROM_GRAL,ASISTENCIA,SIT_FIN,SIT_FIN_R,COD_MEN
0,2022,1,9,LICEO POLITECNICO ARICA,15,AYP,151,15101,ARICA,151,...,15101,ARICA,400,410,41001,"6,6",98,P,P,0
1,2022,1,9,LICEO POLITECNICO ARICA,15,AYP,151,15101,ARICA,151,...,15101,ARICA,400,410,41001,0,0,Y,T,0
2,2022,1,9,LICEO POLITECNICO ARICA,15,AYP,151,15101,ARICA,151,...,15101,ARICA,400,410,41001,0,0,Y,Y,0
3,2022,1,9,LICEO POLITECNICO ARICA,15,AYP,151,15101,ARICA,151,...,15101,ARICA,400,410,41001,0,0,Y,T,0
4,2022,1,9,LICEO POLITECNICO ARICA,15,AYP,151,15101,ARICA,151,...,15101,ARICA,400,410,41001,0,0,Y,Y,0



Rendimiento_2023_2024.csv -> (5, 37)
['\ufeffAGNO', 'RBD', 'DGV_RBD', 'NOM_RBD', 'COD_REG_RBD', 'NOM_REG_RBD_A', 'COD_PRO_RBD', 'COD_COM_RBD', 'NOM_COM_RBD', 'COD_DEPROV_RBD', 'NOM_DEPROV_RBD', 'COD_DEPE', 'COD_DEPE2', 'RURAL_RBD', 'ESTADO_ESTAB', 'COD_ENSE', 'COD_ENSE2', 'COD_GRADO', 'LET_CUR', 'COD_JOR', 'COD_TIP_CUR', 'COD_DES_CUR', 'MRUN', 'GEN_ALU', 'FEC_NAC_ALU', 'EDAD_ALU', 'COD_REG_ALU', 'COD_COM_ALU', 'NOM_COM_ALU', 'COD_RAMA', 'COD_SEC', 'COD_ESPE', 'PROM_GRAL', 'ASISTENCIA', 'SIT_FIN', 'SIT_FIN_R', 'COD_MEN']


,﻿AGNO,RBD,DGV_RBD,NOM_RBD,COD_REG_RBD,NOM_REG_RBD_A,COD_PRO_RBD,COD_COM_RBD,NOM_COM_RBD,COD_DEPROV_RBD,...,COD_COM_ALU,NOM_COM_ALU,COD_RAMA,COD_SEC,COD_ESPE,PROM_GRAL,ASISTENCIA,SIT_FIN,SIT_FIN_R,COD_MEN
0,2023,1,9,LICEO POLITECNICO ARICA,15,AYP,151,15101,ARICA,151,...,15101,ARICA,400,410,41001,0,0,Y,Y,0
1,2023,1,9,LICEO POLITECNICO ARICA,15,AYP,151,15101,ARICA,151,...,15101,ARICA,400,410,41001,"5,7",94,P,P,0
2,2023,1,9,LICEO POLITECNICO ARICA,15,AYP,151,15101,ARICA,151,...,15101,ARICA,400,410,41001,0,0,Y,Y,0
3,2023,1,9,LICEO POLITECNICO ARICA,15,AYP,151,15101,ARICA,151,...,15101,ARICA,400,410,41001,"6,7",100,P,P,0
4,2023,1,9,LICEO POLITECNICO ARICA,15,AYP,151,15101,ARICA,151,...,15101,ARICA,400,410,41001,0,0,Y,Y,0



Rendimiento_2024_2025.csv -> (5, 38)
['\ufeffAGNO', 'RBD', 'DGV_RBD', 'NOM_RBD', 'COD_REG_RBD', 'NOM_REG_RBD_A', 'COD_PRO_RBD', 'COD_COM_RBD', 'NOM_COM_RBD', 'COD_DEPROV_RBD', 'NOM_DEPROV_RBD', 'COD_DEPE', 'COD_DEPE2', 'RURAL_RBD', 'ESTADO_ESTAB', 'NOMBRE_SLEP', 'COD_ENSE', 'COD_ENSE2', 'COD_GRADO', 'LET_CUR', 'COD_JOR', 'COD_TIP_CUR', 'COD_DES_CUR', 'MRUN', 'GEN_ALU', 'FEC_NAC_ALU', 'EDAD_ALU', 'COD_REG_ALU', 'COD_COM_ALU', 'NOM_COM_ALU', 'COD_RAMA', 'COD_SEC', 'COD_ESPE', 'PROM_GRAL', 'ASISTENCIA', 'SIT_FIN', 'SIT_FIN_R', 'COD_MEN']


,﻿AGNO,RBD,DGV_RBD,NOM_RBD,COD_REG_RBD,NOM_REG_RBD_A,COD_PRO_RBD,COD_COM_RBD,NOM_COM_RBD,COD_DEPROV_RBD,...,COD_COM_ALU,NOM_COM_ALU,COD_RAMA,COD_SEC,COD_ESPE,PROM_GRAL,ASISTENCIA,SIT_FIN,SIT_FIN_R,COD_MEN
0,2024,1,9,LICEO POLITECNICO ARICA,15,AYP,151,15101,ARICA,151,...,15101,ARICA,400,410,41001,"6,2",99,P,P,0
1,2024,1,9,LICEO POLITECNICO ARICA,15,AYP,151,15101,ARICA,151,...,15101,ARICA,400,410,41001,0,0,Y,T,0
2,2024,1,9,LICEO POLITECNICO ARICA,15,AYP,151,15101,ARICA,151,...,15101,ARICA,400,410,41001,"5,2",85,R,R,0
3,2024,1,9,LICEO POLITECNICO ARICA,15,AYP,151,15101,ARICA,151,...,15101,ARICA,400,410,41001,"5,8",98,P,P,0
4,2024,1,9,LICEO POLITECNICO ARICA,15,AYP,151,15101,ARICA,151,...,15101,ARICA,400,410,41001,0,0,Y,T,0
